In [ ]:
import numpy as np
from pyomo.environ import *
import sys
import itertools
from itertools import product
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from pyomo.opt import ProblemFormat
from IPython.display import Markdown as md

# Helper functions

In [ ]:
# Calculate distance matrices T_S and T_W using the Euclidean distance
def dist(a,b):
    x1,y1 = a
    x2,y2 = b
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

In [ ]:
def plot_model(ax=None):
    if not ax:
        _, ax = plt.subplots()

    # draw lines between waypoints and S
    for d,s,w in product(range(N_d), range(N_s), range(N_w)):
        pos_s = positions_S[s]
        pos_w = positions_w[d][w]
        distance = dist(pos_s, pos_w)
        x = [pos_s[0], pos_w[0]]
        y = [pos_s[1], pos_w[1]]
        ax.plot(x,y, color='k', alpha=0.2)

        alpha=0.7 # SET ALPHA

        x_text = pos_s[0] + alpha*(pos_w[0] - pos_s[0])
        y_text = pos_s[1] + alpha*(pos_w[1] - pos_s[1])
        ax.text(x_text, y_text, f"{distance:.1f}", color='k', alpha=0.4)

    # draw charge station(s)
    for s in range(N_s):
        x_s = positions_S[s][0]
        y_s = positions_S[s][1]
        ax.scatter(x_s, y_s, marker='s', color='k', s=100)

        # label station
        x_text = x_s + 0.1
        y_text = y_s
        ax.text(x_text, y_text, f"$s_{s+1}$", fontsize=15)


    # draw waypoints and lines for each drone
    for d in range(N_d):
        waypoints = positions_w[d]
        x = [w[0] for w in waypoints[:N_w]]
        y = [w[1] for w in waypoints[:N_w]]
        ax.plot(x,y, marker='o', color=w_colors[d], markersize=10, markerfacecolor='white')    

        # label waypoints
        for w in range(N_w):
            x_text = waypoints[w][0]
            y_text = waypoints[w][1] + 0.05
            ax.text(x_text, y_text, f"$w^{d+1}_{w+1}$", color=w_colors[d], fontsize=15)

        # label time between waypoints
        for w_s in range(N_w-1):
            pos_w_s = waypoints[w_s] 
            pos_w_d = waypoints[w_s+1]        
            distance = dist(pos_w_s, pos_w_d)

            alpha = 0.5 # SET ALPHA

            x_text = pos_w_s[0] + alpha*(pos_w_d[0] - pos_w_s[0])
            y_text = pos_w_s[1] + alpha*(pos_w_d[1] - pos_w_s[1]) + 0.05
            ax.text(x_text, y_text, f"{distance:.1f}", color=w_colors[d])

    # add margins to plot
    plt.margins(0.2)
        

# Model definition (single drone)

In [ ]:
# PARAMETERS
N_d = 1 # number of drones
N_w = 5 # number of waypoints
N_s = 2 # number of recharge stations

B_min = 0
B_max = 1
B_start = [1]

m = np.array([1])
r_charge = np.array([0.1])
r_deplete = np.array([0.25])

D_max = (B_max - B_min) / r_charge

In [ ]:
positions_S = [(3,1), (5,-1)] 

positions_w = [
    [(0,0), (2.5,0), (4.5,0), (6,0), (7.5, 0)],
] # d x w

In [ ]:
T_W = [] # d x w_s
for d in range(N_d):
    row = []
    for w_s in range(N_w-1):
        a = positions_w[d][w_s]
        b = positions_w[d][w_s+1]
        row.append(dist(a,b))        
    T_W.append(row)
T_W = np.array(T_W)

T_S = [] # d x s x w
for d in range(N_d):
    matr= []
    for s in range(N_s):
        a = positions_S[s]
        
        row = []
        for w in range(N_w):
            b = positions_w[d][w]
            row.append(dist(a,b))
        matr.append(row)
    T_S.append(matr)
T_S = np.array(T_S)

print("T_S:")
print(T_S)
print()
print("T_W:")
print(T_W)

In [ ]:
_, ax = plt.subplots(figsize=(10,5))
plot_model(ax=ax)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
plt.show()

In [ ]:
model = ConcreteModel()

model.d = RangeSet(0,0) 
model.w = RangeSet(0, N_w-1) 
model.w_s = RangeSet(0, N_w-2) 
model.w_d = RangeSet(1, N_w-1) 
model.s = RangeSet(0, N_s-1)
model.p = RangeSet(0, N_d*(N_w + N_s)-1)

# VARIABLES
model.x = Var(model.d, model.s, model.w_s, domain=Binary)
model.D = Var(model.d, model.s, model.w_s, domain=NonNegativeReals)
model.b_arr = Var(model.d, model.w)
model.b_min = Var(model.d, model.s, model.w_s)
model.b_plus = Var(model.d, model.s, model.w_s)

# CONSTRAINTS
model.at_most_one_station_per_drone = ConstraintList()
for d, w_s in product(model.d, model.w_s):
    model.at_most_one_station_per_drone.add(sum(model.x[d,s,w_s] for s in model.s) <= 1)
    
# battery constraints
model.start_full = ConstraintList()
for d in model.d:
    model.start_full.add(model.b_arr[d,0] == B_start[d])

model.b_arr_calc = ConstraintList()
for d, w_d in product(model.d, model.w_d):
    model.b_arr_calc.add(
        model.b_arr[d,w_d] == sum(model.x[d,s,w_d-1] * (model.b_plus[d,s,w_d-1] - T_S[d,s,w_d] * r_deplete[d]) for s in model.s) + (1-sum(model.x[d,s,w_d-1] for s in model.s))*(model.b_arr[d,w_d-1] - T_W[d,w_d-1]*r_deplete[d])
    )
    
model.b_min_calc = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.b_min_calc.add(
        model.b_min[d,s,w_s] == model.b_arr[d,w_s] - T_S[d,s,w_s] * r_deplete[d]
    )
    
model.b_plus_calc = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.b_plus_calc.add(
        model.b_plus[d,s,w_s] ==  model.b_min[d,s,w_s] + model.D[d,s,w_s] * r_charge[d]
    )
    
model.b_min_llim = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.b_min_llim.add(
        model.b_min[d,s,w_s] * model.x[d,s,w_s] >= B_min
    )
    
model.b_plus_ulim = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.b_plus_ulim.add(
        model.b_plus[d,s,w_s] * model.x[d,s,w_s] <= B_max
    )
    
model.b_arr_llim = ConstraintList()
for d, w in product(model.d, model.w):
    model.b_arr_llim.add(
        model.b_arr[d,w] >= B_min
    )
    
model.D_ulim = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.D_ulim.add(
        model.D[d,s,w_s] <= D_max[d]
    )
    
# OBJECTIVE
def P_direct(d, w_s):
    return (1 - sum(model.x[d,s,w_s] for s in model.s)) * T_W[d,w_s]

def P_via_S(d, w_s):
    return sum(model.x[d,s,w_s] * (T_S[d,s,w_s] + T_S[d,s,w_s+1] + model.D[d,s,w_s]) for s in model.s)

def P(d): 
    return sum(P_direct(d,w_s) + P_via_S(d,w_s) for w_s in model.w_s)

model.execution_time = Objective(
    expr=sum(P(d) for d in model.d),
    sense=minimize
)

In [ ]:
solution = SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver='ipopt', tee=True) 
# solution = SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver='ipopt') 

In [ ]:
print(f"The optimal execution time: {model.execution_time():.2f}s")
charging_time = sum(np.array(model.D[:,:,:]()) * np.array(model.x[:,:,:]()))
print(f"Of this, the drone(s) spent {charging_time:.2f}s charging")
print(f"Which means that the drone(s) spent {model.execution_time() - charging_time:.2f}s flying")

In [ ]:
d = 0

print(f"---- DRONE {d} ----")
for w_s in model.w_s:
    print(f"[w{d},{w_s+1}]")
    print(f"battery at arrival is {model.b_arr[0,w_s]()*100:.1f}%")
    
    if sum(model.x[d,:,w_s]()) == 0:
        # not charging
        depleted = T_W[d,w_s] * r_deplete[d]
        print(f"moving to next waypoint directly, depleting battery by {depleted*100:.1f}%") 
    else:
        for s in model.s:
            if model.x[d,s,w_s]():
                # go charge at charging station s
                depleted = T_S[d,s,w_s] * r_deplete[0]
                print(f"moving to S{s+1} first, depleting battery by {depleted*100:.1f}%")
                print(f"battery before charging at 'S{s+1}' is {model.b_min[d,s,w_s]()*100:.1f}%")
                charging_duration = model.D[d,s,w_s]()
                print(f"charging battery for {charging_duration:.2f}s, charging {charging_duration*r_charge[d]*100:.1f}%")
                print(f"battery after charging at 'S{s+1}' is {model.b_plus[d,s,w_s]()*100:.1f}%")
    print()
w_s = N_w-1
print(f"[w{d},{w_s+1}]")    
print(f"battery at arrival is {model.b_arr[d,N_w-1]()*100:.1f}%")
print()

# Model definition (two drones)

In [ ]:
# PARAMETERS
N_d = 1 # number of drones (must be 2)
N_w = 6 # number of waypoints
N_s = 1 # number of recharge stations

B_min = 0
B_max = 1
B_start = [1]

m = np.array([1])
r_charge = np.array([0.1])
r_deplete = np.array([0.25])

D_max = (B_max - B_min) / r_charge

In [ ]:
positions_S = [(2.5, 1)]

positions_w = [
    [(0,2), (1,2), (2,2), (3,2), (4,2), (5,2)],
    [(0,0), (1,0), (2,0), (3,0), (4,0), (5,0)],
] # d x w
        
T_W = [] # d x w_s
for d in range(N_d):
    row = []
    for w_s in range(N_w-1):
        a = positions_w[d][w_s]
        b = positions_w[d][w_s+1]
        row.append(dist(a,b))        
    T_W.append(row)
T_W = np.array(T_W)

T_S = [] # d x s x w
for d in range(N_d):
    matr= []
    for s in range(N_s):
        a = positions_S[s]
        
        row = []
        for w in range(N_w):
            b = positions_w[d][w]
            row.append(dist(a,b))
        matr.append(row)
    T_S.append(matr)
T_S = np.array(T_S)

print("T_S:")
print(T_S)
print()
print("T_W:")
print(T_W)

In [ ]:
_, ax = plt.subplots(figsize=(10,5))
plot_model(ax=ax)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
plt.show()

In [ ]:
model = ConcreteModel()

model.d = RangeSet(0,0) 
model.s = RangeSet(0, N_s-1)
model.w = RangeSet(0, N_w-1) 
model.w_s = RangeSet(0, N_w-2) 
model.w_d = RangeSet(1, N_w-1) 

# VARIABLES
model.x = Var(model.d, model.s, model.w_s, domain=Binary)
model.D = Var(model.d, model.s, model.w_s, domain=NonNegativeReals)
model.b_arr = Var(model.d, model.w)
model.b_min = Var(model.d, model.s, model.w_s)
model.b_plus = Var(model.d, model.s, model.w_s)

# CONSTRAINTS
model.at_most_one_station_per_drone = ConstraintList()
for d, w_s in product(model.d, model.w_s):
    model.at_most_one_station_per_drone.add(sum(model.x[d,s,w_s] for s in model.s) <= 1)

# battery constraints
model.start_full = ConstraintList()
for d in model.d:
    model.start_full.add(model.b_arr[d,0] == B_start[d])

model.b_arr_calc = ConstraintList()
for d, w_d in product(model.d, model.w_d):
    model.b_arr_calc.add(
        model.b_arr[d,w_d] == sum(model.x[d,s,w_d-1] * (model.b_plus[d,s,w_d-1] - T_S[d,s,w_d] * r_deplete[d]) for s in model.s) + (1-sum(model.x[d,s,w_d-1] for s in model.s))*(model.b_arr[d,w_d-1] - T_W[d,w_d-1]*r_deplete[d])
    )
    
model.b_min_calc = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.b_min_calc.add(
        model.b_min[d,s,w_s] == model.b_arr[d,w_s] - T_S[d,s,w_s] * r_deplete[d]
    )
    
model.b_plus_calc = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.b_plus_calc.add(
        model.b_plus[d,s,w_s] ==  model.b_min[d,s,w_s] + model.D[d,s,w_s] * r_charge[d]
    )
    
model.b_min_llim = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.b_min_llim.add(
        model.b_min[d,s,w_s] * model.x[d,s,w_s] >= B_min
    )
    
model.b_plus_ulim = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.b_plus_ulim.add(
        model.b_plus[d,s,w_s] * model.x[d,s,w_s] <= B_max
    )
    
model.b_arr_llim = ConstraintList()
for d, w in product(model.d, model.w):
    model.b_arr_llim.add(
        model.b_arr[d,w] >= B_min
    )
    
model.D_ulim = ConstraintList()
for d, s, w_s in product(model.d, model.s, model.w_s):
    model.D_ulim.add(
        model.D[d,s,w_s] <= D_max[d]
    )
    
# OBJECTIVE
def P_direct(d, w_s):
    return (1 - sum(model.x[d,s,w_s] for s in model.s)) * T_W[d,w_s]

def P_via_S(d, w_s):
    return sum(model.x[d,s,w_s] * (T_S[d,s,w_s] + T_S[d,s,w_s+1] + model.D[d,s,w_s]) for s in model.s)

def P(d): 
    return sum(P_direct(d,w_s) + P_via_S(d,w_s) for w_s in model.w_s)

model.execution_time = Objective(
    expr=sum(P(d) for d in model.d),
    sense=minimize
)

In [ ]:
solution = SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver='ipopt', tee=True) 
# solution = SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver='ipopt') 

# SolverFactory("apopt", executable="/Users/au13241/apopt.py").solve(model, tee=True)

In [ ]:
print(f"The optimal execution time: {model.execution_time():.2f}s")
charging_time = sum(np.array(model.D[:,:,:]()) * np.array(model.x[:,:,:]()))
print(f"Of this, the drone(s) spent {charging_time:.2f}s charging")
print(f"Which means that the drone(s) spent {model.execution_time() - charging_time:.2f}s flying")

In [ ]:
for d in model.d:
    print(f"---- DRONE {d} ----")
    for w_s in model.w_s:
        print(f"[w{d},{w_s+1}]")
        print(f"battery at arrival is {model.b_arr[0,w_s]()*100:.1f}%")

        if sum(model.x[d,:,w_s]()) == 0:
            # not charging
            depleted = T_W[d,w_s] * r_deplete[d]
            print(f"moving to next waypoint directly, depleting battery by {depleted*100:.1f}%") 
        else:
            for s in model.s:
                if model.x[d,s,w_s]():
                    # go charge at charging station s
                    depleted = T_S[d,s,w_s] * r_deplete[0]
                    print(f"moving to S{s+1} first, depleting battery by {depleted*100:.1f}%")
                    print(f"battery before charging at 'S{s+1}' is {model.b_min[d,s,w_s]()*100:.1f}%")
                    charging_duration = model.D[d,s,w_s]()
                    print(f"charging battery for {charging_duration:.2f}s, charging {charging_duration*r_charge[d]*100:.1f}%")
                    print(f"battery after charging at 'S{s+1}' is {model.b_plus[d,s,w_s]()*100:.1f}%")
        print()
    w_s = N_w-1
    print(f"[w{d},{w_s+1}]")    
    print(f"battery at arrival is {model.b_arr[d,N_w-1]()*100:.1f}%")
    print()